# Data collection of soccer players

## Libraries

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import LabelEncoder as LE
from scipy.stats import norm, skew, kurtosis, anderson, kstest
import seaborn as sns

## Open csv files

In [2]:
age_df = pd.read_csv("AGE.csv")
date_of_birth =  pd.read_csv('players.csv', usecols= ['player_id', 'birth_date'])
mv_df =  pd.read_csv('market_value_df.csv')
player_data =  pd.read_csv('player table189.csv')
stc_data = pd.read_csv('player_season_preprocessed.csv')

### clean age_df data

In [3]:
age_df.loc[age_df['Age'].str.contains('†'), 'Age'] = age_df['Age'].str.replace('†', '')

### clean player_df data

In [4]:
player_df = player_data.drop(player_data.columns[0], axis=1)
player_df = player_df.drop('birth_date', axis=1)

height_median = player_df['height'].median()
# Replace NaN values with the median of the 'height' column
player_df['height'] = player_df['height'].fillna(height_median)

player_df['main_position'].replace(['Attack Left Winger','Attack Centre Forward', 'Attack Right Winger', 'Attack Second Striker'], 'striker', inplace=True)
player_df['main_position'].replace(['Defender Centre Back', 'Defender Right Back', 'Defender Left Back'], 'defender', inplace=True)
player_df['main_position'].replace(['midfield Central Midfield', 'midfield Attacking Midfield', 'midfield Defensive Midfield', 'midfield', 'midfield Left Midfield', 'midfield Right Midfield'], 'midfielder', inplace=True)

player_df['goals_scored'].replace('-', np.nan, inplace=True)
player_df['goals_assisted'].replace('-', np.nan, inplace=True)
player_df['total_appearence'].replace('-', np.nan, inplace=True)
player_df['agent'] = player_df['agent'].replace('', np.nan)

player_df['goals_conceded'] = np.where(player_df['main_position'] != 'Goalkeeper', 'not defined',
                       np.where(player_df['goals_conceded'] == '-', np.nan, player_df['goals_conceded']))

player_df['clean_sheets'] = np.where(player_df['main_position'] != 'Goalkeeper', 'not defined',
                       np.where(player_df['clean_sheets'] == '-', np.nan, player_df['clean_sheets']))

In [5]:
print(age_df[age_df['Age'].str.contains('†', na=False)])
print(player_df['main_position'].unique())

Empty DataFrame
Columns: [season, Age, player_id, season_short]
Index: []
['Goalkeeper' 'defender' 'midfielder' 'striker']


### clean stc_df data

In [6]:
stc_df = stc_data

stc_df['Appearances'].replace('-', np.nan, inplace=True)
stc_df['PPG'].replace('-', np.nan, inplace=True)
stc_df['PPG'].replace('0,00', 0, inplace=True)
stc_df['Goals'].replace('-', np.nan, inplace=True)
stc_df['Own goals'].replace('-', np.nan, inplace=True)
stc_df['Substitutions on'].replace('-', np.nan, inplace=True)
stc_df['Substitutions off'].replace('-', np.nan, inplace=True)
stc_df['Yellow cards'].replace('-', np.nan, inplace=True)
stc_df['Second yellow cards'].replace('-', np.nan, inplace=True)
stc_df['Red cards'].replace('-', np.nan, inplace=True)
stc_df['Assists'].replace('-', np.nan, inplace=True)
stc_df['Penalty goals'].replace('-', np.nan, inplace=True)
stc_df['Minutes per goal'].replace('-', np.nan, inplace=True)

stc_df['Season']= stc_df['Season'].astype(str)

ppg_median = stc_df['PPG'].median()
# Replace NaN values with the median of the 'PPG' column
stc_df['PPG'] = stc_df['PPG'].fillna(ppg_median)

In [7]:
def year_transformer(raw_year):
    year = raw_year.split('/')[0]
    if len(year) == 2:
        if int(year) <=22:
            return '20' + year
        else:
            return '19' + year
    elif len(year) == 4:
        return year

In [8]:
stc_df['Season'] = stc_df['Season'].apply(lambda x: year_transformer(x))

In [9]:
stc_df.rename(columns={'Season':'season'}, inplace=True)

In [10]:
stc_df['season'] = stc_df['season'].astype(int)

In [11]:
stc_df.head()

,player_id,season,Age,Competition,Club,club_id,position_group,New_season,Squad,Appearances,...,Own goals,Assists,Penalty goals,Yellow cards,Second yellow cards,Red cards,Substitutions on,Substitutions off,Minutes played,Minutes per goal
0,2857,2017,32,Premier League,Leicester City,1003,Goalkeeper,False,10,2,...,0,0,0,0,0,0,0,0,180,0
1,16911,2017,30,Premier League,Leicester City,1003,Goalkeeper,False,33,33,...,1,0,0,3,0,0,0,1,2966,0
2,56810,2017,28,Premier League,Leicester City,1003,midfield,False,15,12,...,0,1,0,3,0,0,3,4,761,0
3,61560,2017,27,Premier League,Leicester City,1003,midfield,False,35,34,...,0,7,0,5,0,1,4,10,2543,1272
4,65467,2017,29,Premier League,Leicester City,1003,midfield,False,29,19,...,0,0,0,0,0,0,2,7,1414,471


In [12]:
print(stc_df.dtypes)

player_id                int64
season                   int64
Age                      int64
Competition             object
Club                    object
club_id                  int64
position_group          object
New_season                bool
Squad                    int64
Appearances              int64
club_clean_sheet         int64
player_clean_sheet       int64
Goals conceded           int64
PPG                    float64
Goals                    int64
Own goals                int64
Assists                  int64
Penalty goals            int64
Yellow cards             int64
Second yellow cards      int64
Red cards                int64
Substitutions on         int64
Substitutions off        int64
Minutes played           int64
Minutes per goal         int64
dtype: object


In [13]:
stc_df.shape

(7432, 25)

In [14]:
np.shape(mv_df)

(12594, 3)

In [15]:
np.shape(stc_df)

(7432, 25)

In [16]:
final_df = stc_df.merge(mv_df, on=['player_id', 'season'], how='left') 

In [17]:
final_df = final_df.dropna(subset=['market_value'])

In [31]:
final_df.head()

,player_id,season,Age,Competition,Club,club_id,position_group,New_season,Squad,Appearances,...,Assists,Penalty goals,Yellow cards,Second yellow cards,Red cards,Substitutions on,Substitutions off,Minutes played,Minutes per goal,market_value
0,2857,2017,32,Premier League,Leicester City,1003,2,False,10,2,...,0,0,0,0,0,0,0,180,0,1000000.0
3,61560,2017,27,Premier League,Leicester City,1003,3,False,35,34,...,7,0,5,0,1,4,10,2543,1272,10000000.0
4,65467,2017,29,Premier League,Leicester City,1003,3,False,29,19,...,0,0,0,0,0,2,7,1414,471,10000000.0
6,174915,2017,29,Premier League,Leicester City,1003,0,False,19,12,...,1,0,0,0,0,10,1,239,239,18000000.0
7,177907,2017,24,Premier League,Leicester City,1003,1,False,38,38,...,3,0,7,0,0,0,0,3420,1710,25000000.0


In [19]:
final_df.shape

(4089, 26)

In [29]:
le = LE()
le.fit(final_df['position_group'])
final_df['position_group'] = le.transform(final_df['position_group'])

In [30]:
final_df['position_group']

0       2
3       3
4       3
6       0
7       1
       ..
7619    0
7620    1
7621    1
7622    3
7625    0
Name: position_group, Length: 4089, dtype: int64

In [36]:
final_df.to_csv('final_df.csv')

In [22]:
Goalkeeper_df = stc_df[stc_df['position_group'] == 'Goalkeeper'] 
striker_df = stc_df[stc_df['position_group'] == 'striker'] 
defender_df = stc_df[stc_df['position_group'] == 'defender'] 
midfielder_df = stc_df[stc_df['position_group'] == 'midfielder']

In [23]:
np.unique(Goalkeeper_df['position_group'])

array(['Goalkeeper'], dtype=object)

In [24]:
Goalkeeper_df.shape

(794, 25)

In [25]:
Goalkeeper_df.drop(['Goals', 'Own goals', 'Substitutions on', 'Substitutions off',
                   'Yellow cards', 'Second yellow cards', 'Red cards', 'Minutes played',
                   ], inplace=True, axis=1)

/tmp/ipykernel_48530/4223849242.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Goalkeeper_df.drop(['Goals', 'Own goals', 'Substitutions on', 'Substitutions off',


In [26]:

Goalkeeper_df_m = Goalkeeper_df.merge(mv_df, on=['player_id', 'season'], how='left') 

In [27]:
Goalkeeper_df_m.shape

(801, 18)

In [28]:
Goalkeeper_df_m = Goalkeeper_df_m.dropna(subset=['market_value'])
Goalkeeper_df_m 

,player_id,season,Age,Competition,Club,club_id,position_group,New_season,Squad,Appearances,club_clean_sheet,player_clean_sheet,Goals conceded,PPG,Assists,Penalty goals,Minutes per goal,market_value
0,2857,2017,32,Premier League,Leicester City,1003,Goalkeeper,False,10,2,8,0,6,1.50,0,0,0,1000000.0
3,16911,2021,34,Premier League,Leicester City,1003,Goalkeeper,False,38,37,7,7,58,1.32,0,0,0,4000000.0
9,14044,2019,35,Premier League,Liverpool FC,31,Goalkeeper,False,2,0,17,0,0,0.00,0,0,0,250000.0
15,340918,2020,21,Premier League,Liverpool FC,31,Goalkeeper,False,15,2,12,1,1,1.50,0,0,0,2500000.0
17,71271,2021,34,Premier League,Liverpool FC,31,Goalkeeper,False,5,0,21,0,0,0.00,0,0,0,1000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
783,79045,2020,29,Serie A,Spezia Calcio,3522,Goalkeeper,False,20,7,5,1,12,1.43,0,0,0,2000000.0
785,79045,2021,30,Serie A,Spezia Calcio,3522,Goalkeeper,False,36,7,8,0,19,0.57,0,0,0,850000.0
792,208166,2018,20,Serie A,ACF Fiorentina,430,Goalkeeper,False,20,3,10,1,2,1.33,0,0,0,10000000.0
795,448632,2019,19,Ligue 1,Stade Reims,1421,Goalkeeper,True,1,0,12,0,0,0.00,0,0,0,350000.0
